In [14]:
#!env/bin/python3
''' Cobus Smit 2021
    started from 
    https://www.mattbutton.com/2019/01/24/how-to-scrape-yahoo-finance-and-extract-fundamental-stock-market-data-using-python-lxml-and-pandas/
    https://stackoverflow.com/questions/56528631/is-there-a-version-of-selenium-webdriver-that-is-not-detectable/56529616#56529616


'''
import os
from datetime import datetime
import time
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By


def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }

    return requests.get(url, headers=headers)


def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)

    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64

    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want

    ## tree = html.fromstring( get_page(url).content )

    # Replace get_page[request] above with get_page_webdriver
    tree = html.fromstring( get_page_webdriver(url) )

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0

    df = parse_rows(table_rows)
    df = clean_data(df)

    return df


def get_page_webdriver(url):
    tstart = time.time()
    options = webdriver.ChromeOptions() 
    options.add_argument("start-maximized")
    options.add_argument("--headless")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    # Find path to chromedriver
    for fname in [ r'C:\Utility\BrowserDrivers\chromedriver.exe',
                   r'C:\Users\Killer\Desktop\Phyton\chromedriver.exe',
                   r'/usr/bin/chromedriver',
                 ]:
        if os.path.isfile(fname):
            browser = webdriver.Chrome(options=options, executable_path = fname)
            break
    else:
        print("Error in get_page_webdriver could not find chromedriver ?")
        exit(1)
    print(f"get_page_webdriver - loading {url}")
    browser.get(url)
    print(f"                   - scroll down ...", end="")
    browser.execute_script("window.scrollTo(0,300)")
    time.sleep(1)
    print(f" click expand ...", end="")
    click_expand = browser.find_element_by_xpath('//*[@id="Col1-1-Financials-Proxy"]/section/div[2]/button')
    click_expand.click()

    # Fetch the page that we're going to parse, using the request headers
    # defined above
    ## page = requests.get(url, headers)
    pageContent = browser.page_source
    print(f" got {len(pageContent):,} bytes in {time.time()-tstart:.1f}s.")
    return pageContent


def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)

    df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
    df_balance_sheet = df_balance_sheet.set_index('Date')

    df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
    df_income_statement = df_income_statement.set_index('Date')

    df_cash_flow = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    df_cash_flow = df_cash_flow.set_index('Date')

    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
    df_joined.insert(1, 'Symbol', symbol)
    return df_joined


def scrape_multi(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

print("Loaded python functions.")


Loaded python functions.


In [15]:
''' Use functions above to retrieve stock symbols'''
symbols = ['KIO.JO', 'SOL.JO']
df_combined = scrape_multi(symbols)
df_combined




Attempting to scrape data for KIO.JO
get_page_webdriver - loading https://finance.yahoo.com/quote/KIO.JO/balance-sheet?p=KIO.JO


KeyboardInterrupt: 

In [13]:
''' Write pandas dataframe to excel '''
date = datetime.today().strftime('%Y-%m-%d')
filename = f'Yahoo-Finance-Scrape2-{date}.xlsx'
writer = pd.ExcelWriter(filename)
df_combined.to_excel(writer)
writer.save()
print(f"The END. saved {filename}")

The END. saved Yahoo-Finance-Scrape2-2021-01-23.xlsx
